Book Einführung in Optimirungsmodelle from Nathan Sudermann-Merx page 112


In [1]:
import polars as pl
import pyoframe as pf
from pyoframe import sum_by

In [2]:
init_values = pl.read_csv('./initial_numbers.csv', has_header=False, new_columns=['row', 'column', 'digit'])
init_values

row,column,digit
i64,i64,i64
1,1,8
2,3,3
2,4,6
3,2,7
3,5,9
…,…,…
8,3,8
8,4,5
8,8,1


In [3]:
one_to_nine = pl.DataFrame({"digit": range(1, 10)})
one_to_nine

digit
i64
1
2
3
4
5
6
7
8
9


In [4]:
grid = one_to_nine.join(one_to_nine, how="cross").rename(
    {"digit": "row", "digit_right": "column"}
)
grid

row,column
i64,i64
1,1
1,2
1,3
1,4
1,5
…,…
9,5
9,6
9,7


In [5]:
m = pf.Model('sudoku_binary_pyoframe', solver='highs', use_var_names=True)
m.Y = pf.Variable(grid.join(one_to_nine, how="cross"), vtype=pf.VType.BINARY) # row x column x digit
m.Y

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


<Variable name=Y size=729 dimensions={'row': 9, 'column': 9, 'digit': 9}>
[1,1,1]: Y[1,1,1]
[1,1,2]: Y[1,1,2]
[1,1,3]: Y[1,1,3]
[1,1,4]: Y[1,1,4]
[1,1,5]: Y[1,1,5]
[1,1,6]: Y[1,1,6]
[1,1,7]: Y[1,1,7]
[1,1,8]: Y[1,1,8]
[1,1,9]: Y[1,1,9]
[1,2,1]: Y[1,2,1]
[1,2,2]: Y[1,2,2]
[1,2,3]: Y[1,2,3]
[1,2,4]: Y[1,2,4]
[1,2,5]: Y[1,2,5]
[1,2,6]: Y[1,2,6]

In [6]:
m.given_values = m.Y.drop_unmatched() == pf.Set(init_values)
m.given_values

<Constraint name=given_values sense='=' size=21 dimensions={'row': 9, 'column': 9, 'digit': 9} terms=42>
[1,1,8]: Y[1,1,8] = 1
[2,3,3]: Y[2,3,3] = 1
[2,4,6]: Y[2,4,6] = 1
[3,2,7]: Y[3,2,7] = 1
[3,5,9]: Y[3,5,9] = 1
[3,7,2]: Y[3,7,2] = 1
[4,2,5]: Y[4,2,5] = 1
[4,6,7]: Y[4,6,7] = 1
[5,5,4]: Y[5,5,4] = 1
[5,6,5]: Y[5,6,5] = 1
[5,7,7]: Y[5,7,7] = 1
[6,4,1]: Y[6,4,1] = 1
[6,8,3]: Y[6,8,3] = 1
[7,3,1]: Y[7,3,1] = 1
[7,8,6]: Y[7,8,6] = 1

In [7]:
m.one_per_row = sum_by(["digit", "row"], m.Y) == 1
m.one_per_row

<Constraint name=one_per_row sense='=' size=81 dimensions={'row': 9, 'digit': 9} terms=810>
[1,1]: Y[1,1,1] + Y[1,2,1] + Y[1,3,1] + Y[1,4,1] + Y[1,5,1] + Y[1,6,1] + Y[1,7,1] + Y[1... = 1
[1,2]: Y[1,1,2] + Y[1,2,2] + Y[1,3,2] + Y[1,4,2] + Y[1,5,2] + Y[1,6,2] + Y[1,7,2] + Y[1... = 1
[1,3]: Y[1,1,3] + Y[1,2,3] + Y[1,3,3] + Y[1,4,3] + Y[1,5,3] + Y[1,6,3] + Y[1,7,3] + Y[1... = 1
[1,4]: Y[1,1,4] + Y[1,2,4] + Y[1,3,4] + Y[1,4,4] + Y[1,5,4] + Y[1,6,4] + Y[1,7,4] + Y[1... = 1
[1,5]: Y[1,1,5] + Y[1,2,5] + Y[1,3,5] + Y[1,4,5] + Y[1,5,5] + Y[1,6,5] + Y[1,7,5] + Y[1... = 1
[1,6]: Y[1,1,6] + Y[1,2,6] + Y[1,3,6] + Y[1,4,6] + Y[1,5,6] + Y[1,6,6] + Y[1,7,6] + Y[1... = 1
[1,7]: Y[1,1,7] + Y[1,2,7] + Y[1,3,7] + Y[1,4,7] + Y[1,5,7] + Y[1,6,7] + Y[1,7,7] + Y[1... = 1
[1,8]: Y[1,1,8] + Y[1,2,8] + Y[1,3,8] + Y[1,4,8] + Y[1,5,8] + Y[1,6,8] + Y[1,7,8] + Y[1... = 1
[1,9]: Y[1,1,9] + Y[1,2,9] + Y[1,3,9] + Y[1,4,9] + Y[1,5,9] + Y[1,6,9] + Y[1,7,9] + Y[1... = 1
[2,1]: Y[2,1,1] + Y[2,2,1] + Y[2,3,1] + Y[2,4,1] + Y[

In [8]:
m.one_per_column = sum_by(["digit", "column"], m.Y) == 1
m.one_per_column

<Constraint name=one_per_column sense='=' size=81 dimensions={'column': 9, 'digit': 9} terms=810>
[1,1]: Y[1,1,1] + Y[2,1,1] + Y[3,1,1] + Y[4,1,1] + Y[5,1,1] + Y[6,1,1] + Y[7,1,1] + Y[8... = 1
[1,2]: Y[1,1,2] + Y[2,1,2] + Y[3,1,2] + Y[4,1,2] + Y[5,1,2] + Y[6,1,2] + Y[7,1,2] + Y[8... = 1
[1,3]: Y[1,1,3] + Y[2,1,3] + Y[3,1,3] + Y[4,1,3] + Y[5,1,3] + Y[6,1,3] + Y[7,1,3] + Y[8... = 1
[1,4]: Y[1,1,4] + Y[2,1,4] + Y[3,1,4] + Y[4,1,4] + Y[5,1,4] + Y[6,1,4] + Y[7,1,4] + Y[8... = 1
[1,5]: Y[1,1,5] + Y[2,1,5] + Y[3,1,5] + Y[4,1,5] + Y[5,1,5] + Y[6,1,5] + Y[7,1,5] + Y[8... = 1
[1,6]: Y[1,1,6] + Y[2,1,6] + Y[3,1,6] + Y[4,1,6] + Y[5,1,6] + Y[6,1,6] + Y[7,1,6] + Y[8... = 1
[1,7]: Y[1,1,7] + Y[2,1,7] + Y[3,1,7] + Y[4,1,7] + Y[5,1,7] + Y[6,1,7] + Y[7,1,7] + Y[8... = 1
[1,8]: Y[1,1,8] + Y[2,1,8] + Y[3,1,8] + Y[4,1,8] + Y[5,1,8] + Y[6,1,8] + Y[7,1,8] + Y[8... = 1
[1,9]: Y[1,1,9] + Y[2,1,9] + Y[3,1,9] + Y[4,1,9] + Y[5,1,9] + Y[6,1,9] + Y[7,1,9] + Y[8... = 1
[2,1]: Y[1,2,1] + Y[2,2,1] + Y[3,2,1] + Y[4,2,1

In [9]:
m.one_per_cell = sum_by(["row", "column"], m.Y) == 1
m.one_per_cell

<Constraint name=one_per_cell sense='=' size=81 dimensions={'row': 9, 'column': 9} terms=810>
[1,1]: Y[1,1,1] + Y[1,1,2] + Y[1,1,3] + Y[1,1,4] + Y[1,1,5] + Y[1,1,6] + Y[1,1,7] + Y[1... = 1
[1,2]: Y[1,2,1] + Y[1,2,2] + Y[1,2,3] + Y[1,2,4] + Y[1,2,5] + Y[1,2,6] + Y[1,2,7] + Y[1... = 1
[1,3]: Y[1,3,1] + Y[1,3,2] + Y[1,3,3] + Y[1,3,4] + Y[1,3,5] + Y[1,3,6] + Y[1,3,7] + Y[1... = 1
[1,4]: Y[1,4,1] + Y[1,4,2] + Y[1,4,3] + Y[1,4,4] + Y[1,4,5] + Y[1,4,6] + Y[1,4,7] + Y[1... = 1
[1,5]: Y[1,5,1] + Y[1,5,2] + Y[1,5,3] + Y[1,5,4] + Y[1,5,5] + Y[1,5,6] + Y[1,5,7] + Y[1... = 1
[1,6]: Y[1,6,1] + Y[1,6,2] + Y[1,6,3] + Y[1,6,4] + Y[1,6,5] + Y[1,6,6] + Y[1,6,7] + Y[1... = 1
[1,7]: Y[1,7,1] + Y[1,7,2] + Y[1,7,3] + Y[1,7,4] + Y[1,7,5] + Y[1,7,6] + Y[1,7,7] + Y[1... = 1
[1,8]: Y[1,8,1] + Y[1,8,2] + Y[1,8,3] + Y[1,8,4] + Y[1,8,5] + Y[1,8,6] + Y[1,8,7] + Y[1... = 1
[1,9]: Y[1,9,1] + Y[1,9,2] + Y[1,9,3] + Y[1,9,4] + Y[1,9,5] + Y[1,9,6] + Y[1,9,7] + Y[1... = 1
[2,1]: Y[2,1,1] + Y[2,1,2] + Y[2,1,3] + Y[2,1,4] + 

In [10]:
cell_to_box = grid.with_columns(
    box=((pl.col("row") - 1) // 3) * 3 + (pl.col("column") - 1) // 3 + 1
)
cell_to_box

row,column,box
i64,i64,i64
1,1,1
1,2,1
1,3,1
1,4,2
1,5,2
…,…,…
9,5,8
9,6,8
9,7,9


In [11]:
m.one_per_box = m.Y.map(cell_to_box) == 1
m.one_per_box

<Constraint name=one_per_box sense='=' size=81 dimensions={'digit': 9, 'box': 9} terms=810>
[1,1]: Y[1,1,1] + Y[1,2,1] + Y[1,3,1] + Y[2,1,1] + Y[2,2,1] + Y[2,3,1] + Y[3,1,1] + Y[3... = 1
[1,2]: Y[1,4,1] + Y[1,5,1] + Y[1,6,1] + Y[2,4,1] + Y[2,5,1] + Y[2,6,1] + Y[3,4,1] + Y[3... = 1
[2,1]: Y[1,1,2] + Y[1,2,2] + Y[1,3,2] + Y[2,1,2] + Y[2,2,2] + Y[2,3,2] + Y[3,1,2] + Y[3... = 1
[2,2]: Y[1,4,2] + Y[1,5,2] + Y[1,6,2] + Y[2,4,2] + Y[2,5,2] + Y[2,6,2] + Y[3,4,2] + Y[3... = 1
[3,1]: Y[1,1,3] + Y[1,2,3] + Y[1,3,3] + Y[2,1,3] + Y[2,2,3] + Y[2,3,3] + Y[3,1,3] + Y[3... = 1
[3,2]: Y[1,4,3] + Y[1,5,3] + Y[1,6,3] + Y[2,4,3] + Y[2,5,3] + Y[2,6,3] + Y[3,4,3] + Y[3... = 1
[4,1]: Y[1,1,4] + Y[1,2,4] + Y[1,3,4] + Y[2,1,4] + Y[2,2,4] + Y[2,3,4] + Y[3,1,4] + Y[3... = 1
[4,2]: Y[1,4,4] + Y[1,5,4] + Y[1,6,4] + Y[2,4,4] + Y[2,5,4] + Y[2,6,4] + Y[3,4,4] + Y[3... = 1
[5,1]: Y[1,1,5] + Y[1,2,5] + Y[1,3,5] + Y[2,1,5] + Y[2,2,5] + Y[2,3,5] + Y[3,1,5] + Y[3... = 1
[5,2]: Y[1,4,5] + Y[1,5,5] + Y[1,6,5] + Y[2,4,5] + Y[

In [12]:
m.optimize()

MIP  has 345 rows; 730 cols; 3282 nonzeros; 729 integer variables (729 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+00]
  Cost   [0e+00, 0e+00]
  Bound  [1e+00, 1e+00]
  RHS    [0e+00, 0e+00]
Presolving model
229 rows, 248 cols, 994 nonzeros  0s
176 rows, 218 cols, 870 nonzeros  0s
169 rows, 213 cols, 872 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   169 rows
   213 cols (213 binary, 0 integer, 0 implied int., 0 continuous, 0 domain fixed)
   872 nonzeros

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue

In [13]:
m.Y.solution.filter(pl.col('solution') == 1).pivot("column", index=["row"], values="digit")

row,1,2,3,4,5,6,7,8,9
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,8,1,2,7,5,3,6,4,9
2,9,4,3,6,8,2,1,7,5
3,6,7,5,4,9,1,2,8,3
4,1,5,4,2,3,7,8,9,6
5,3,6,9,8,4,5,7,2,1
6,2,8,7,1,6,9,5,3,4
7,5,2,1,9,7,4,3,6,8
8,4,3,8,5,2,6,9,1,7
9,7,9,6,3,1,8,4,5,2


In [14]:
m.write('sudoku_model.mps')

Writing the model to sudoku_model.mps
